## ตัวอย่างการทำ Topic Modeling
เพื่อทำสรุปหัวข้อเอกสาร




ติดตั้ง Libraries สำหรับจัดการข้อมูลประเภทข้อความภาษาอังกฤษ

In [ ]:
!pip install pyLDAvis
!pip install --upgrade pyLDAvis

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import  word_tokenize
from nltk.corpus import stopwords
import spacy
import string
import re
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import  LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.lda_model


In [19]:
import warnings
warnings.filterwarnings("ignore")

ทำการเชื่อม Google Drive

In [20]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


ระบุ Folder ใน Drive ที่ต้องการใช้

In [5]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

อ่านข้อมูลจากไฟล์

In [ ]:
import pandas as pd

data = pd.read_csv("SampleTopics.csv",encoding='latin-1')
print(data )

# Data Preprocessing  สร้างเป็นฟังก์ชันเพื่อเรียกใช้ซ้ำ

In [22]:
def data_preprocessing(cleaned_data):
    # กำจัด URL
    cleaned_data = " ".join(re.sub(r"https?://[A-Za-z0-9./]+", '', w ) for w in cleaned_data.split())

    # แปลงข้อมูลเป็นตัวพิมพ์เล็ก ... lower case
    cleaned_data = " ".join(w.lower() for w in cleaned_data.split())

    # Lemmatization ด้วย Spacy
    nlp = spacy.load("en_core_web_sm")
    cleaned_data =  " ".join([w.lemma_ for w in nlp(cleaned_data) if w.lemma_ != "-PRON-"])

    # กำจัด stop words
    stop = stopwords.words('english')
    cleaned_data = " ".join(w for w in word_tokenize(cleaned_data) if w not in stop)

     # กำจัด stop words
    stop = stopwords.words('english')
    cleaned_data = " ".join(w for w in word_tokenize(cleaned_data) if w not in ('study'))

    # ตัดเครื่องหมายวรรคตอน ... remove punctuation
    cleaned_data =  " ".join([w for w in cleaned_data.split() if w not in list(string.punctuation)])

    # กำจัดตัวเลข
    cleaned_data = " ".join(re.sub(r"[0-9]+", '', w ) for w in cleaned_data.split())

    # กำจัดข้อความหลัง ' เครื่องหมาย aposophi
    cleaned_data = " ".join(re.sub(r"(')[A-Za-z]+", '', w ) for w in cleaned_data.split())

    return cleaned_data

เรียกใช้งานฟังก์ชัน data_preprocessing

In [ ]:
data['Content'] = data['Content'].apply(data_preprocessing)

# แสดงผลข้อมูลหลังจาก Data Preprocessing
print(data)

# สร้าง document term matrix

In [14]:

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(data['Content'])
tf_feature_names = tf_vectorizer.get_feature_names_out()
print(tf_feature_names)

['adaptation' 'algorithm' 'ancient' 'animal' 'argue' 'aristotle' 'atom'
 'behavior' 'biology' 'bond' 'chemical' 'chemistry' 'compound' 'computer'
 'concept' 'concern' 'datum' 'discipline' 'element' 'form' 'formal'
 'fundamental' 'group' 'history' 'include' 'interact' 'kingdom'
 'knowledge' 'make' 'matter' 'molecule' 'natural' 'practice' 'problem'
 'process' 'program' 'reaction' 'science' 'scientific' 'structure'
 'substance' 'theory' 'use' 'zoology']


สร้าง object for LDA model using gensim library

In [10]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([ (feature_names[i] + " (" + str(topic[i].round(2)) + ")")
          for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(str(doc_index) + ". " + documents[doc_index])


In [ ]:
#@title Set topic modeling algorithm arguments

no_topics = 6 #@param {type:"integer"}

# Run LDA
lda_model = LatentDirichletAllocation(n_components=no_topics,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

print("LDA Topics")
display_topics(lda_H, lda_W, tf_feature_names, data['Content'], no_top_words=3, no_top_documents=no_topics)
# Visualization can be displayed in the notebook
pyLDAvis.enable_notebook()

pyLDAvis_data = pyLDAvis.lda_model.prepare(lda_model, tf, tf_vectorizer)

# Visualization can be displayed in the notebook
pyLDAvis.display(pyLDAvis_data)
